In [2]:
pip install pdfplumber

  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.0-py3-none-win_amd64.whl.metadata (48 kB)
Using cached pdfplumber-0.11.4-py3-none-any.whl (59 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-4.30.0-py3-none-win_amd64.whl (2.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pdfplumber
def extract_with_pdf(path):
    text =""
    with pdfplumber.open(path) as pdfreader:
        for page in pdfreader.pages:
            text+=page.extract_text() or ""
    return text


In [19]:
text_extracted=extract_with_pdf(r"D:\waste for git\Generative_AI\Rag\attention all you need.pdf")
#print(text_extracted)

In [13]:
pip install Langchain


Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [31]:
# dividing into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,         # Maximum size of each chunk
        chunk_overlap=50,       # Overlap between chunks
        length_function=len,    # Function to calculate length
        separators=["\n\n", "\n", ". ", " ", ""]  # Possible split points
    )
    chunks = text_splitter.split_text(text)
    return chunks

# Example usage
text_chunks = get_text_chunks(text_extracted)
print(text_chunks)
print(len(text_extracted))

#get_text_chunks(text_extracted)

['Attention Is All You Need\nAshishVaswani∗ NoamShazeer∗ NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch\navaswani@google.com noam@google.com nikip@google.com usz@google.com\nLlionJones∗ AidanN.Gomez∗ † ŁukaszKaiser∗\nGoogleResearch UniversityofToronto GoogleBrain\nllion@google.com aidan@cs.toronto.edu lukaszkaiser@google.com\nIlliaPolosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThedominantsequencetransductionmodelsarebasedoncomplexrecurrentor', 'convolutionalneuralnetworksthatincludeanencoderandadecoder. Thebest\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbasedsolelyonattentionmechanisms,dispensingwithrecurrenceandconvolutions\nentirely. Experiments on two machine translation tasks show these models to\nbesuperiorinqualitywhilebeingmoreparallelizableandrequiringsignificantly', 'less time to train. Our model achieves 28.4 BLEU on the WMT 2

In [33]:
pip install faiss-cpu


   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.8/14.9 MB 11.2 MB/s eta 0:00:02
   ----------- ---------------------------- 4.2/14.9 MB 11.4 MB/s eta 0:00:01
   ----------------- ---------------------- 6.6/14.9 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 8.9/14.9 MB 11.3 MB/s eta 0:00:01
   ------------------------------ --------- 11.3/14.9 MB 11.2 MB/s eta 0:00:01
   ------------------------------------ --- 13.6/14.9 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------  14.7/14.9 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 9.3 MB/s eta 0:00:00


In [34]:
# embeddings and storing
def get_vector_store(text_chunks):
    embeddings=HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
    vector_store=FAISS.from_texts(text_chunks,embedding=embeddings)
    vector_store.save_local("faiss_index")
    
get_vector_store(text_chunks)

In [26]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama

def create_qa_chain():
    prompt_template = """
    Use the following pieces of context to answer the question. If you cannot find the answer in the context, respond with "The answer is not available in the context." Do not make up or infer any information that is not explicitly stated in the context.

    Context: {context}

    Question: {question}

    Instructions:
    1. Only use information from the provided context
    2. If the exact answer is in the context, provide it
    3. If the answer is not in the context, say "The answer is not available in the context"
    4. Keep the answer concise and relevant

    Answer:
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # llm = Ollama(model="llama3.1", temperature=0.3)  # Load the Llama model

    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    # vectorstore = FAISS.load_local("faiss_index", embeddings)

    # retriever = vectorstore.as_retriever(
    #     search_type="similarity",  # Use similarity search to retrieve relevant chunks
    #     search_kwargs={
    #         "k": 5,                   # Number of relevant chunks to retrieve
    #         "fetch_k": 15             # Total chunks to fetch
    #     }
    # )

    # Create a QA chain that combines LLM with the retriever
    # qa_chain = RetrievalQA.from_chain_type(
    #     llm=llm,
    #     chain_type="stuff",  # Use the "stuff" type for this chain
    #     retriever=retriever,
    #     chain_type_kwargs={
    #         "prompt": PROMPT,
    #     },
    #     return_source_documents=True  # Optionally return source documents for transparency
    # )
    
    return True

# Example usage
qa_chain = create_qa_chain()


c:\Users\Mohammed Riyaz\anaconda3\envs\rag\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Memory

class Memory:
    def __init__(self):
        self.conversations = []

    def add_conversation(self, user_input, bot_response):
        self.conversations.append({'user': user_input, 'bot': bot_response})

    def get_context(self):
        return "\n".join([f"User: {conv['user']}\nBot: {conv['bot']}" for conv in self.conversations])

# Example usage in handle_user_input
memory = Memory()

def handle_user_input(user_question):
    context = memory.get_context()
    try:
        qa_chain = create_qa_chain()  # Create the QA chain
        response = qa_chain.invoke({
            "query": f"{context}\nUser: {user_question}"  # Include memory context
        })
        
        answer = response.get('result', '').strip()  # Extract the answer
        memory.add_conversation(user_question, answer)  # Store the conversation

        st.write("Reply: ", answer)

    except Exception as e:
        st.error(f"An error occurred: {str(e)}")
        st.write("Reply: The answer is not available in the context.")


In [ ]:
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        with pdfplumber.open(pdf) as pdf_reader:
            for page in pdf_reader.pages:
                text += page.extract_text() or ""
                tables = page.extract_tables()
                for table in tables:
                    for row in table:
                        # Filter out None values and join the rest into a string
                        filtered_row = [cell for cell in row if cell is not None]
                        if filtered_row:  # Check if the filtered row is not empty
                            text += " | ".join(filtered_row) + "\n"  # Join table rows into text

In [1]:
#for csv
import csv
def get_csv_text(csv_file):
    text=""
    with open(csv_file,mode='r',encoding='utf-8') as file:
        csv_reader=csv.reader(file)
        for row in csv_reader:
            filtered_row=[cell for cell in row if  cell]
            if filtered_row:
                text+="|".join(filtered_row) +"\n"
    return text
        

In [4]:
text_extract=get_csv_text('D:\waste for git\Generative_AI\Chat_With_Pdf\Local_Model\IPL_Ball_by_Ball_2008_2022.csv')
#print(text_extract)

In [13]:
pip install openpyxl


  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [17]:
import pandas as pd
from langchain_core.documents import Document
def get_excel_text(excel_files):
    text = ""
    lis=[]
    for excel_file in excel_files: 
        excel_data = pd.read_excel(excel_file, sheet_name=None)  # Load all sheets
        for sheet_name, sheet_data in excel_data.items():
            text += f"Sheet: {sheet_name}\n"
            for _, row in sheet_data.iterrows():
                filtered_row = [str(cell) for cell in row if pd.notna(cell)]
                if filtered_row:
                    text += " | ".join(filtered_row) + "\n"
            text += "\n"  #
            lis.append(Document(page_content = text,metadata={'source' : f"Sheet {sheet_name}"}))
    return lis

In [18]:
#excel_extract=get_excel_text([r"D:\waste for git\Generative_AI\Chat_With_Pdf\Local_Model\file_new 1.xlsx"])

print(excel_extract)

[Document(metadata={'source': 'Sheet Sheet1'}, page_content='Sheet: Sheet1\nif not mystring.strip()\n | This code snippet is used to check if the string myString is either empty or contains only whitespace characters. In data cleaning, this check can be helpful to identify and handle missing or irrelevant data, which is often necessary before feeding the data into a machine learning model.  Here\'s how it works:  1.myString.strip(): The strip() method removes any leading or trailing whitespace characters from myString. If the result is an empty string, it indicates that the original string was either completely empty or consisted only of whitespace characters.  2.if not myString.strip()::  The not operator negates the result of myString.strip(). If myString.strip() returns an empty string (which is considered False in Python), the if condition becomes True. This means that if myString is either empty or contains only whitespace characters, the code inside the if block will be executed.

In [32]:
rc = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)

In [33]:
docs = rc.split_documents(documents=excel_extract)

In [68]:
# docs

In [36]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Mohammed Riyaz\anaconda3\envs\rag\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [48]:
fa = FAISS.from_documents(documents=docs,embedding=embeddings)

In [64]:
r = fa.as_retriever(search_type="mmr",
                search_kwargs={'k': 10, 'lambda_mult': 0.25})

In [65]:
len(excel_extract[0].page_content)

46342

In [67]:
data  = r.invoke("Colorma")
# len(data),data
data

[Document(metadata={'source': 'Sheet Sheet1'}, page_content='colorama_init()'),
 Document(metadata={'source': 'Sheet Sheet1'}, page_content='black("BLACK")\nred("RED")\ngreen("GREEN")\nyellow("YELLOW")\nblue("BLACK")\nmagenta("MAGENTA")\ncyan("CYAN")\ngray("GRAY")'),
 Document(metadata={'source': 'Sheet Sheet1'}, page_content='an empty string colors for accumulating color-formatted text in each iteration of the outer loop.  3. Inner Loop: for j in range(5):: Loops 5 times for each iteration of the outer loop. The loop'),
 Document(metadata={'source': 'Sheet Sheet1'}, page_content='def colored(r, g, b, text):\n    return f"\\033[38;2;{r};{g};{b}m{text}\\033"'),
 Document(metadata={'source': 'Sheet Sheet1'}, page_content='from termcolor import colored'),
 Document(metadata={'source': 'Sheet Sheet1'}, page_content='command-line interfaces. 2. Printing Foreground Colors: print(color("This text is red", Colors.red)): color("This text is red", Colors.red): Uses the color function to set the 

In [1]:
pip install python-pptx python-docx

  Using cached python_pptx-1.0.2-py3-none-any.whl.metadata (2.5 kB)
  Using cached python_docx-1.1.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached XlsxWriter-3.2.0-py3-none-any.whl.metadata (2.6 kB)
Using cached python_pptx-1.0.2-py3-none-any.whl (472 kB)
Using cached python_docx-1.1.2-py3-none-any.whl (244 kB)
Using cached XlsxWriter-3.2.0-py3-none-any.whl (159 kB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
from pptx import Presentation
from langchain.docstore.document import Document

def get_ppt_text(ppt_file):
    """Extract text from PowerPoint files efficiently."""
    documents = []
    prs = Presentation(ppt_file)
    
    all_slide_texts = []  # Accumulate all slide texts here
    for i, slide in enumerate(prs.slides):
        slide_text_parts = []  # Accumulate text per slide
        for shape in slide.shapes:
            if shape.has_text_frame:  # Optimized check for text
                slide_text_parts.append(shape.text)  # Add text content
        
        slide_text = "\n".join(slide_text_parts)
        all_slide_texts.append(f"Slide {i+1}:\n{slide_text}")
        
        # Create a Document for each slide
        documents.append(Document(page_content=slide_text, metadata={'source': f'PowerPoint Slide {i+1}'}))
    
    # Combine all slide texts into a single string
    full_text = "\n\n".join(all_slide_texts)
    
    return full_text, documents


In [21]:
get_ppt_text("ppt_report.pptx")

('Slide 1:\n\n\n\n\nInternship Presentation\nSubmitted by:\nAnkita\nIntern – Software Development Trainee\nCO20309\nCSE 4th year\nManager – Mr. Rajul Goyal\xa0\n\n\n\n\nSlide 2:\n\nTesting Xperts\xa0(Tx) proudly stands as a global leader in Digital\xa0Assurance and Quality Engineering, firmly positioned among the five\xa0largest providers worldwide. With headquarters strategically located in\xa0Pennsylvania, USA, and London, UK, Testing Xperts(Tx) maintain a dynamic network of\xa0offices and delivery centers across the United States, Canada, the United\xa0Kingdom,\xa0the Netherlands, South Africa,\xa0UAE, India, and Singapore.\x0b\x0bAt\xa0Testing Xperts, our expertise shines through a comprehensive\xa0spectrum of Digital Assurance and Quality Engineering services that\xa0encompass QA/Test Advisory, as well as Functional and Non-Functional\xa0testing. These services are impeccably tailored to meet the demands of\xa0Next Gen technologies. Our specialization extends seamlessly to\xa0Auto

In [3]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [7]:
def get_image_text(image_files):
    """Extract text from image files using OCR."""
    text = ""
    documents = []
    for image_file in image_files:
        try:
            image = Image.open(image_file)
            image_text = pytesseract.image_to_string(image)
            cleaned_text = ' '.join(image_text.split())  # Remove extra whitespace
            text += cleaned_text + "\n"
            documents.append(Document(page_content=cleaned_text, metadata={'source': f"Image: {image_file}"}))
        except Exception as e:
            print(f"Error processing image {image_file}: {str(e)}")
    return text, documents

In [8]:
get_image_text("Screenshot (13).png")

Error processing image S: name 'Image' is not defined
Error processing image c: name 'Image' is not defined
Error processing image r: name 'Image' is not defined
Error processing image e: name 'Image' is not defined
Error processing image e: name 'Image' is not defined
Error processing image n: name 'Image' is not defined
Error processing image s: name 'Image' is not defined
Error processing image h: name 'Image' is not defined
Error processing image o: name 'Image' is not defined
Error processing image t: name 'Image' is not defined
Error processing image  : name 'Image' is not defined
Error processing image (: name 'Image' is not defined
Error processing image 1: name 'Image' is not defined
Error processing image 3: name 'Image' is not defined
Error processing image ): name 'Image' is not defined
Error processing image .: name 'Image' is not defined
Error processing image p: name 'Image' is not defined
Error processing image n: name 'Image' is not defined
Error processing image g: na

('', [])

In [10]:
pip install pytesseract


Note: you may need to restart the kernel to use updated packages.


In [14]:
import tesseract
print(tesseract --version)

ModuleNotFoundError: No module named 'tesseract'

In [11]:
import pytesseract
from PIL import Image

# Load image
image = Image.open('Screenshot (13).png')

# Perform OCR
text = pytesseract.image_to_string(image)

print(text)


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.